<a href="https://colab.research.google.com/github/wizard339/article_finder/blob/main/article_finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib.request
import urllib.parse
import argparse
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import time

In [2]:
BASE_URL = 'http://export.arxiv.org/api/query?search_query='
PREFIX = {'Title': 'ti',
          'Author': 'au',
          'Abstract': 'abs',
          'Comment': 'co',
          'Journal Reference': 'jr',
          'Subject Category': 'cat',
          'Report Number': 'rn',
          'ID': 'id',
          'All': 'all'}
TIME_PERIOD = {'Day': datetime.timedelta(1),
               'Week': datetime.timedelta(7),
               'Mounth': datetime.timedelta(30),
               'Year': datetime.timedelta(365)}
CURRENT_DATE = datetime.datetime.now()

# input_keywords = input('Please enter the keywords or search phrases separated by commas: ')
input_keywords = 'reinforcement AND learning'

In [3]:
CURRENT_DATE - TIME_PERIOD['Week']

datetime.datetime(2022, 9, 3, 7, 20, 57, 885159)

In [12]:
def make_query(base_url=BASE_URL, prefix=PREFIX['All'], start=0, keywords=input_keywords, time_period=TIME_PERIOD['Week']):
    """нужно ввести счетчик и проверку на значение тега updated в entry"""
    # the starting date from which we receive the data
    starting_date = CURRENT_DATE - time_period
    # encoding of the search query
    keywords = urllib.parse.quote_plus(keywords)

    cols = ['updated', 'title', 'summary', 'author', 'link']
    articles = pd.DataFrame(columns=cols)
    start_counter = 0
    
    while True:
      url = f'{base_url}{prefix}:{keywords}&start={start_counter}&max_results=30000&sortBy=lastUpdatedDate&sortOrder=descending'
      print(url)
      req = urllib.request.Request(url)
      with urllib.request.urlopen(req) as response:
          if response.status==200:
              soup = BeautifulSoup(response, 'xml')
          else:
              raise ValueError('Please, check the correctness of the request')
              break
      date_on_the_page = datetime.datetime(*[int(i) for i in soup.entry.updated.string.split('T')[0].split('-')])
      print(date_on_the_page)
      if date_on_the_page >= starting_date:
        print(soup.entry.updated)
        for tag in soup.find_all('entry'):
            updated_to_df = pd.to_datetime(tag.updated.string.split('T')[0])
            authors_list = ", ".join([n.string for n in tag.find_all('name')])
            link_to_df = tag.find(title='pdf').get('href')
            row_to_concat = pd.DataFrame({'updated': updated_to_df,
                                          'title': tag.title,
                                          'summary': tag.summary,
                                          'author': authors_list,
                                          'link': link_to_df}, index=tag.id)
            articles = pd.concat([articles, row_to_concat])
      else:
        break
      start_counter += 1
      time.sleep(3)

    return articles

In [14]:
articles = make_query()

http://export.arxiv.org/api/query?search_query=all:reinforcement+AND+learning&start=0&max_results=30000&sortBy=lastUpdatedDate&sortOrder=descending


HTTPError: ignored

In [ ]:
datetime.datetime(2022,8,31)

datetime.datetime(2022, 8, 31, 0, 0)

In [6]:
articles.sample(10)

,updated,title,summary,author,link
http://arxiv.org/abs/2201.08299v3,2022-09-02,Goal-Conditioned Reinforcement Learning: Probl...,Goal-conditioned reinforcement learning (GCR...,"Minghuan Liu, Menghui Zhu, Weinan Zhang",http://arxiv.org/pdf/2201.08299v3
http://arxiv.org/abs/2208.04822v2,2022-08-29,Generalized Reinforcement Learning: Experience...,Learning a control policy capable of adaptin...,"Po-Hsiang Chiu, Manfred Huber",http://arxiv.org/pdf/2208.04822v2
http://arxiv.org/abs/2203.07889v4,2022-08-30,Comparing Two Samples Through Stochastic Domin...,Non-deterministic measurements are common in...,"Etor Arza, Josu Ceberio, Ekhiñe Irurozki, Arit...",http://arxiv.org/pdf/2203.07889v4
http://arxiv.org/abs/2207.06559v2,2022-09-01,Scalable Model-based Policy Optimization for D...,Reinforcement learning algorithms require a ...,"Yali Du, Chengdong Ma, Yuchen Liu, Runji Lin, ...",http://arxiv.org/pdf/2207.06559v2
http://arxiv.org/abs/2209.02166v1,2022-09-05,To Compute or not to Compute? Adaptive Smart S...,We consider a network of smart sensors for e...,"Luca Ballotta, Giovanni Peserico, Francesco Za...",http://arxiv.org/pdf/2209.02166v1
http://arxiv.org/abs/2208.11639v2,2022-08-26,Oracle-free Reinforcement Learning in Mean-Fie...,We consider online reinforcement learning in...,"Muhammad Aneeq uz Zaman, Alec Koppel, Sujay Bh...",http://arxiv.org/pdf/2208.11639v2
http://arxiv.org/abs/2208.12622v1,2022-08-26,Play with Emotion: Affect-Driven Reinforcement...,This paper introduces a paradigm shift by vi...,"Matthew Barthet, Ahmed Khalifa, Antonios Liapi...",http://arxiv.org/pdf/2208.12622v1
http://arxiv.org/abs/2201.13170v3,2022-09-01,Cooperative Online Learning in Stochastic and ...,We study cooperative online learning in stoc...,"Tal Lancewicki, Aviv Rosenberg, Yishay Mansour",http://arxiv.org/pdf/2201.13170v3
http://arxiv.org/abs/2208.08133v2,2022-09-05,Metric Residual Networks for Sample Efficient ...,Goal-conditioned reinforcement learning (GCR...,"Bo Liu, Yihao Feng, Qiang Liu, Peter Stone",http://arxiv.org/pdf/2208.08133v2
http://arxiv.org/abs/2208.12345v1,2022-08-25,Light-weight probing of unsupervised represent...,Unsupervised visual representation learning ...,"Wancong Zhang, Anthony GX-Chen, Vlad Sobal, Ya...",http://arxiv.org/pdf/2208.12345v1


In [7]:
articles.updated.value_counts()

2022-09-01    1298
2022-08-26    1180
2022-08-30    1062
2022-08-22     760
2022-08-31     708
2022-08-29     708
2022-08-23     702
2022-08-25     649
2022-09-05     637
2022-09-02     531
2022-08-24     531
2022-09-07     430
2022-09-03     413
2022-09-06     407
2022-08-28     354
2022-08-27     354
2022-08-19     253
2022-08-21     252
2022-09-04     230
2022-08-18     153
2022-08-20     122
2022-09-08      66
Name: updated, dtype: int64

In [8]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11800 entries, http://arxiv.org/abs/2209.03887v1 to http://arxiv.org/abs/2204.05893v2
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   updated  11800 non-null  datetime64[ns]
 1   title    11800 non-null  object        
 2   summary  11800 non-null  object        
 3   author   11800 non-null  object        
 4   link     11800 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 553.1+ KB
